Prerequisites:

`./AppData/Local/Programs/Python/Python310/Scripts/pip install plotly nbformat`

In [8]:
import numpy as np
import pandas as pd
from scipy.linalg import lstsq
import plotly.express as px

In [9]:
path = r"C:/Users/sulta/git/cone-operator-lab/data/eigenvalues/box_a1.0_b1.5_c2.3_N20000.txt"
eigs_all = np.loadtxt(path, ndmin=1)
eigs_all = np.sort(eigs_all[eigs_all > 0.0])

In [10]:
def compute_A2(eigs, f, *, tmax_factor=500.0, m_grid=151, diagnostics=False):
    """
    Compute A2_hat for a given eigenvalue list and f.
    If diagnostics=True, also return cond(X) and relative residual.
    """
    lamN = eigs[-1]
    t_min = f / lamN
    t_max = tmax_factor / lamN
    if t_min >= t_max:
        return (np.nan, np.nan, np.nan) if diagnostics else np.nan

    # log-spaced t grid
    t = np.geomspace(t_min, t_max, m_grid)

    # Z(t) = sum_k exp(-λ_k t)  (vectorized)
    Z = np.exp(-np.outer(eigs, t)).sum(axis=0)

    # Design matrix: [t^-3/2, t^-1, t^-1/2, 1]
    X = np.column_stack([t**(-1.5), t**(-1.0), t**(-0.5), np.ones_like(t)])

    # least squares (stable, uses LAPACK)
    coef, *_ = lstsq(X, Z)

    A2_hat = (coef[2] * (4*np.pi)**1.5) / (4*np.pi**3)

    if not diagnostics:
        return A2_hat

    condX = np.linalg.cond(X)
    rel_res = np.linalg.norm(X @ coef - Z) / max(np.linalg.norm(Z), 1e-300)
    return A2_hat, condX, rel_res

In [11]:
def scan_A2_grid(eigs_all, n_steps, factors, *, tmax_factor=500.0, m_grid=151, diagnostics=False):
    """
    Compute results for all (N,f) once and return a DataFrame.
    Columns:
      - always: N, f, A2_hat
      - if diagnostics: condX, rel_residual
    """
    rows = []
    for N in n_steps:
        eigs = eigs_all[:N]
        for f in factors:
            out = compute_A2(eigs, f, tmax_factor=tmax_factor, m_grid=m_grid, diagnostics=diagnostics)
            if diagnostics:
                A2_hat, condX, rel_res = out
            else:
                A2_hat = out
                condX, rel_res = np.nan, np.nan

            if np.isnan(A2_hat):
                continue

            if diagnostics:
                rows.append((N, f, A2_hat, condX, rel_res))
            else:
                rows.append((N, f, A2_hat))

    if diagnostics:
        return pd.DataFrame(rows, columns=["N", "f", "A2_hat", "condX", "rel_residual"])
    return pd.DataFrame(rows, columns=["N", "f", "A2_hat"])

In [12]:
n_steps = [2000, 5000, 10000]
factors = np.arange(10, 620, 2)
m_grid = 151
A2_true = (1.0 + 1.5 + 2.3) / (4 * np.pi**2)

tmax_factor = 500.0

In [13]:
# 1) compute once
dfA2 = scan_A2_grid(
    eigs_all=eigs_all,
    n_steps=n_steps,
    factors=factors,
    tmax_factor=tmax_factor,
    m_grid=m_grid,
    diagnostics=False   # diagnostics optional
)

# 2) derive error once (reused everywhere)
dfA2["err_pct"] = np.abs(dfA2["A2_hat"] / A2_true - 1.0) * 100.0

In [14]:
fig = px.line(
    dfA2, x="f", y="err_pct", color="N",
    log_y=True,
    labels={"f": "tMin factor (f/lambda_N)", "err_pct": "relative error of A2 (%)"},
    title="A2 reconstruction error vs f"
)
fig.update_layout(
    width=900,
    height=550,
    margin=dict(l=80, r=40, t=80, b=70)
)
fig.update_traces(mode="lines+markers", hovertemplate="N=%{legendgroup}<br>f=%{x}<br>err=%{y:.4g}%<extra></extra>")
fig.show()

In [15]:
fstar_df = (
    dfA2.loc[dfA2.groupby("N")["err_pct"].idxmin(), ["N", "f", "A2_hat", "err_pct"]]
      .rename(columns={"f": "f_star", "A2_hat": "A2_hat_at_fstar", "err_pct": "rel_error_pct"})
      .sort_values("N")
)

display(fstar_df)

,N,f_star,A2_hat_at_fstar,rel_error_pct
29,2000,68,0.121381,0.168428
373,5000,266,0.121557,0.023767
493,10000,16,0.121303,0.232683
